# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [107]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

import pprint

import json
import time

# Configure gmaps
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [108]:
file = pd.read_csv("WeatherPy.csv")
file.dropna(inplace = True) 
file.head()


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Bluff,-46.6000,168.3333,57.79,61,100,16.51,NZ,1643863850
1,1,Punta Arenas,-53.1500,-70.9167,42.91,70,40,3.44,CL,1643863855
2,2,Jamestown,42.0970,-79.2353,38.52,71,100,4.61,US,1643863825
3,3,Mariehamn,60.0973,19.9348,19.26,93,100,1.14,AX,1643864261
4,4,Airai,-8.9266,125.4092,73.09,62,93,5.77,TL,1643863946


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [109]:
# Store latitude and longitude in locations
locations = file[["Lat", "Lng"]]

# Fill NaN values and convert to float
rating = file["Humidity"]

In [110]:
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=rating, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [9]:
mask = ((file["Max Temp"] > 70) & (file["Max Temp"] < 80))
df10 = file.loc[mask,:]

mask2 = df10["Wind Speed"] < 10
df11 = df10.loc[mask2,:]

mask3 = df11["Cloudiness"] == 0
df12 = df11.loc[mask3,:]
df12.reset_index(drop=True, inplace=True)
df12.shape


(4, 10)

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [10]:
df12

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,139,Comodoro Rivadavia,-45.8667,-67.5000,73.26,38,0,5.75,AR,1643864059
1,287,Acajutla,13.5928,-89.8275,73.80,80,0,4.97,SV,1643864060
2,440,Le Vauclin,14.5452,-60.8388,75.85,88,0,6.91,MQ,1643864383
3,477,Barra de Tecoanapa,16.5167,-98.7500,73.26,76,0,5.84,MX,1643864395


In [74]:
#https://towardsdatascience.com/how-to-use-the-google-places-api-for-location-analysis-and-more-17e48f8f25b1


hotels1 = []
params = {}
  
endpoint_url = "https://maps.googleapis.com/maps/api/place/textsearch/json?query=hotels&location=-45.8667,-67.5000&radius=5000&key="
         
res = requests.get(endpoint_url, params = params)
results =  json.loads(res.content)
hotels1.extend(results['results'])
time.sleep(2)
while "next_page_token" in results:
     params['pagetoken'] = results['next_page_token'],
     res = requests.get(endpoint_url, params = params)
     results = json.loads(res.content)
     hotels1.extend(results['results'])
     time.sleep(2)

In [84]:
comodoro_rivadavia = hotels1[0]["name"]
comodoro_rivadavia

'WAM Hotel Patagónico'

In [76]:
hotels2 = []
params = {}
  
endpoint_url = "https://maps.googleapis.com/maps/api/place/textsearch/json?query=hotels&location=13.5928,-89.8275&radius=5000&key="
         
res = requests.get(endpoint_url, params = params)
results =  json.loads(res.content)
hotels2.extend(results['results'])
time.sleep(2)
while "next_page_token" in results:
     params['pagetoken'] = results['next_page_token'],
     res = requests.get(endpoint_url, params = params)
     results = json.loads(res.content)
     hotels2.extend(results['results'])
     time.sleep(2)

In [85]:
acajutla = hotels2[0]["name"]
acajutla

'Royal Decameron Salinitas'

In [78]:
hotels3 = []
params = {}
  
endpoint_url = "https://maps.googleapis.com/maps/api/place/textsearch/json?query=hotels&location=14.5452,-60.8388&radius=5000&key="
         
res = requests.get(endpoint_url, params = params)
results =  json.loads(res.content)
hotels3.extend(results['results'])
time.sleep(2)
while "next_page_token" in results:
     params['pagetoken'] = results['next_page_token'],
     res = requests.get(endpoint_url, params = params)
     results = json.loads(res.content)
     hotels3.extend(results['results'])
     time.sleep(2)

In [86]:
le_vauclin = hotels3[0]["name"]
le_vauclin

'Karibea Sainte-Luce Hotel'

In [80]:
hotels4 = []
params = {}
  
endpoint_url = "https://maps.googleapis.com/maps/api/place/textsearch/json?query=hotels&location=16.5167,-98.7500&radius=5000&key="
         
res = requests.get(endpoint_url, params = params)
results =  json.loads(res.content)
hotels4.extend(results['results'])
time.sleep(2)
while "next_page_token" in results:
     params['pagetoken'] = results['next_page_token'],
     res = requests.get(endpoint_url, params = params)
     results = json.loads(res.content)
     hotels4.extend(results['results'])
     time.sleep(2)

In [87]:
barra_de_tecoanapa = hotels4[0]["name"]
barra_de_tecoanapa

'Meson Casa de Piedra'

In [104]:
hotel = ["WAM Hotel Patagónico","Royal Decameron Salinitas","Karibea Sainte-Luce Hotel","Meson Casa de Piedra"]
df55 = pd.DataFrame(hotel, columns = ["Hotel Name"])
df55

result10 = pd.concat([df12, df55], axis=1)
result10


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,139,Comodoro Rivadavia,-45.8667,-67.5000,73.26,38,0,5.75,AR,1643864059,WAM Hotel Patagónico
1,287,Acajutla,13.5928,-89.8275,73.80,80,0,4.97,SV,1643864060,Royal Decameron Salinitas
2,440,Le Vauclin,14.5452,-60.8388,75.85,88,0,6.91,MQ,1643864383,Karibea Sainte-Luce Hotel
3,477,Barra de Tecoanapa,16.5167,-98.7500,73.26,76,0,5.84,MX,1643864395,Meson Casa de Piedra


In [106]:
hotel_df5 = result10
hotel_df5

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name,Hotel Name
0,139,Comodoro Rivadavia,-45.8667,-67.5000,73.26,38,0,5.75,AR,1643864059,WAM Hotel Patagónico,WAM Hotel Patagónico
1,287,Acajutla,13.5928,-89.8275,73.80,80,0,4.97,SV,1643864060,Royal Decameron Salinitas,Royal Decameron Salinitas
2,440,Le Vauclin,14.5452,-60.8388,75.85,88,0,6.91,MQ,1643864383,Karibea Sainte-Luce Hotel,Karibea Sainte-Luce Hotel
3,477,Barra de Tecoanapa,16.5167,-98.7500,73.26,76,0,5.84,MX,1643864395,Meson Casa de Piedra,Meson Casa de Piedra


In [111]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df5[["Lat", "Lng"]]

In [113]:
hotel_info

['\n<dl>\n<dt>Name</dt><dd>WAM Hotel Patagónico</dd>\n<dt>City</dt><dd>Comodoro Rivadavia</dd>\n<dt>Country</dt><dd>AR</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Royal Decameron Salinitas</dd>\n<dt>City</dt><dd>Acajutla</dd>\n<dt>Country</dt><dd>SV</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Karibea Sainte-Luce Hotel</dd>\n<dt>City</dt><dd>Le Vauclin</dd>\n<dt>Country</dt><dd>MQ</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Meson Casa de Piedra</dd>\n<dt>City</dt><dd>Barra de Tecoanapa</dd>\n<dt>Country</dt><dd>MX</dd>\n</dl>\n']

In [112]:
# Add marker layer ontop of heat map


# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [117]:
fig2 = gmaps.figure()
locations2 = file[["Lat", "Lng"]]
rating2 = file["Humidity"]
# Create heat layer
heat_layer = gmaps.heatmap_layer(locations2, weights=rating2, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)
symbol_layer = gmaps.symbol_layer(
            locations, info_box_content=hotel_info)

# Add layer
fig2.add_layer(heat_layer)
fig2.add_layer(symbol_layer)


# Display figure
fig2

Figure(layout=FigureLayout(height='420px'))